Neural Networks


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential, save_model, load_model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import joblib
import matplotlib.pyplot as plt
from sklearn.inspection import permutation_importance as sklearn_permutation_importance
   
# Read the CSV file into a pandas DataFrame
data = pd.read_csv('C:/Users/maxva/OneDrive - Tilburg University/Msc. Data Science/Master Thesis/Data/merged_train_sameday_real.csv')
test_data = pd.read_csv('C:/Users/maxva/OneDrive - Tilburg University/Msc. Data Science/Master Thesis/Data/merged_test_sameday_real.csv')
columns_to_remove = ['volume', 'option_price', 'open_interest']
test_data = test_data.drop(columns=columns_to_remove)
data = data.drop(columns=columns_to_remove)
test_data_with_all_predictions = test_data.copy() 

# Make a copy for predictions
test_data_with_all_predictions = test_data.copy()

option_columns = [
    'impl_volatility',
    'cp_flag',
    'stock_price',
    'moneyness',
    'time_to_expiry',
    'strike_price',
    'delta',
    'gamma',
    'vega',
    'theta',
    'rf',
    'iv_ahbs',
    'iv_ahbs_error',
    'iv_bs',
    'iv_bs_error',  # Adjusted from a duplicate iv_ahbs_error
    'iv_cw',
    'iv_cw_error'
]

option_only = data[option_columns]
test_data_with_all_predictions = test_data_with_all_predictions[option_columns]
print(option_only.head())


   impl_volatility  cp_flag  stock_price  moneyness  time_to_expiry  \
0         0.210270        1      115.545   0.970966              24   
1         0.208124        1      115.545   0.962875              24   
2         0.205474        1      115.545   0.954917              24   
3         0.278442        0      115.545   1.100429              24   
4         0.242212        0      115.545   1.050409              24   

   strike_price     delta     gamma       vega      theta      rf   iv_ahbs  \
0         119.0  0.329402  0.057909  10.772000 -16.612410  0.0245  0.346996   
1         120.0  0.273000  0.053728   9.892975 -15.153390  0.0245  0.346099   
2         121.0  0.220664  0.048533   8.819048 -13.381580  0.0245  0.345399   
3         105.0 -0.075725  0.017194   4.246497  -9.138635  0.0245  0.386757   
4         110.0 -0.186759  0.037192   7.995862 -15.079300  0.0245  0.365726   

   iv_ahbs_error     iv_bs  iv_bs_error     iv_cw  iv_cw_error  
0       0.136726  0.353798     0.

In [33]:
test_data_with_all_predictions.describe()

,impl_volatility,cp_flag,stock_price,moneyness,time_to_expiry,strike_price,delta,gamma,vega,theta,...,iv_bs_pred_NN4,iv_bs_corrected_NN4,iv_ahbs_pred_NN3,iv_ahbs_corrected_NN3,iv_ahbs_pred_NN4,iv_ahbs_corrected_NN4,iv_cw_pred_NN3,iv_cw_corrected_NN3,iv_cw_pred_NN4,iv_cw_corrected_NN4
count,1.089824e+06,1.089824e+06,1.089824e+06,1.089824e+06,1.089824e+06,1.089824e+06,1.089824e+06,1.089824e+06,1.089824e+06,1.089824e+06,...,1.089824e+06,1.089824e+06,1.089824e+06,1.089824e+06,1.089824e+06,1.089824e+06,1.089824e+06,1.089824e+06,1.089824e+06,1.089824e+06
mean,3.537372e-01,5.384209e-01,2.168925e+02,1.020033e+00,7.300243e+01,2.147561e+02,5.406997e-02,3.874991e-02,2.547875e+01,-3.387311e+01,...,-2.615054e-03,3.564132e-01,-2.164857e-03,3.558459e-01,-1.970897e-03,3.556519e-01,-2.434737e-03,3.559069e-01,-3.603944e-03,3.570761e-01
std,1.505677e-01,4.985219e-01,4.091524e+02,1.356553e-01,5.502698e+01,4.113761e+02,3.056829e-01,5.054576e-02,5.188064e+01,7.601737e+01,...,1.473609e-01,1.473609e-01,1.304187e-01,1.472153e-01,1.304020e-01,1.470581e-01,1.249892e-01,1.472917e-01,1.248199e-01,1.470375e-01
min,2.641200e-02,0.000000e+00,4.240000e+00,8.000000e-01,2.000000e+01,3.000000e+00,-9.152470e-01,6.000000e-06,1.042100e-02,-1.691377e+03,...,-2.110648e+00,7.020296e-02,-2.004421e+00,1.823548e-02,-2.017903e+00,7.051928e-02,-1.853428e+00,8.636025e-02,-1.885655e+00,6.258065e-02
25%,2.453760e-01,0.000000e+00,5.413500e+01,9.312373e-01,3.100000e+01,5.500000e+01,-2.140933e-01,8.446000e-03,5.972642e+00,-2.855597e+01,...,-7.802840e-02,2.501075e-01,-6.741001e-02,2.496532e-01,-6.711460e-02,2.497281e-01,-6.210580e-02,2.496547e-01,-6.340673e-02,2.510063e-01
50%,3.201395e-01,1.000000e+00,1.165650e+02,9.918776e-01,5.100000e+01,1.150000e+02,9.269950e-02,2.093600e-02,1.196315e+01,-1.209609e+01,...,2.983008e-02,3.239680e-01,2.455792e-02,3.234262e-01,2.487505e-02,3.232279e-01,2.649692e-02,3.233944e-01,2.531405e-02,3.245431e-01
75%,4.289585e-01,1.000000e+00,2.024950e+02,1.080526e+00,1.000000e+02,2.000000e+02,3.259722e-01,4.957800e-02,2.491818e+01,-5.561454e+00,...,1.036906e-01,4.318265e-01,8.839633e-02,4.311001e-01,8.839913e-02,4.309427e-01,8.502727e-02,4.312781e-01,8.371254e-02,4.324269e-01
max,2.712715e+00,1.000000e+00,3.727041e+03,1.600000e+00,2.400000e+02,4.600000e+03,7.801590e-01,1.321314e+00,1.139503e+03,-2.919800e-02,...,2.835951e-01,2.464446e+00,4.517831e-01,2.562475e+00,4.044157e-01,2.552457e+00,2.906757e-01,2.527477e+00,3.406985e-01,2.494889e+00


In [ ]:
###########################################
# PART 1: NEURAL NETWORK MODEL DEFINITION
###########################################

def create_nn_model(architecture_type, input_dim):

    model = Sequential()
    
    # Define architecture based on specifications
    if architecture_type == 'NN3':
        # Three hidden layers with 32, 16, and 8 neurons
        model.add(Dense(128, activation='sigmoid', input_dim=input_dim))
        model.add(Dense(64, activation='sigmoid'))
        model.add(Dense(32, activation='sigmoid'))
        model.add(Dense(1))
    
    elif architecture_type == 'NN4':
        # Four hidden layers with 32, 16, 8, and 4 neurons
        model.add(Dense(128, activation='sigmoid', input_dim=input_dim))
        model.add(Dense(64, activation='sigmoid'))
        model.add(Dense(32, activation='sigmoid'))
        model.add(Dense(16, activation='sigmoid'))
        model.add(Dense(1))
    
    else:
        raise ValueError("Invalid architecture type. Choose from 'NN3' or 'NN4'.")
    
    # Compile model with Adam optimizer for faster training
    model.compile(optimizer=Adam(learning_rate=0.01), loss='mean_squared_error')
    
    return model

def train_and_evaluate_model(model, X_train_scaled, y_train, X_test_scaled, y_test, epochs=100):

    # Early stopping with optimized parameters
    callbacks = [
        EarlyStopping(
            monitor='loss',
            patience=10,
            restore_best_weights=True,
            verbose=1
        ),
        ReduceLROnPlateau(
            monitor='loss',
            factor=0.5,
            patience=5,
            min_lr=1e-5,
            verbose=1
        )
    ]
    
    # Train model with larger batch size for better speed
    history = model.fit(
        X_train_scaled, y_train,
        epochs=epochs,
        batch_size=128,
        validation_split=0.0,
        callbacks=callbacks,
        verbose=1
    )
    
    # Evaluate model
    y_pred = model.predict(X_test_scaled, batch_size=128, verbose=0)
    mse = mean_squared_error(y_test, y_pred)
    
    return history, mse


In [ ]:
###########################################
# PART 2: DATA PREPARATION
###########################################

def prepare_data(option_only):

    # Prepare features and target variables
    feature_columns = [col for col in option_only.columns if col not in 
                      ['iv_bs_error', 'iv_ahbs', "iv_ahbs_error", "iv_bs", 
                       "iv_cw", "iv_cw_error", "impl_volatility", "moneyness_category", 'ID']]
    
    # Instead of using train_test_split, use your predefined sets
    X_train = option_only[feature_columns]  # Features from your training set
    X_test = test_data[feature_columns]  # Features from your test set

    # Target variables for each error type
    y_bs_train = option_only['iv_bs_error']  
    y_bs_test = test_data['iv_bs_error']  

    y_ahbs_train = option_only['iv_ahbs_error']  
    y_ahbs_test = test_data['iv_ahbs_error'] 

    y_cw_train = option_only['iv_cw_error']  
    y_cw_test = test_data['iv_cw_error'] 
    # Initialize a StandardScaler to normalize the features
    scaler = StandardScaler()
    
    # Fit the scaler on the training data and transform both training and testing data
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Create dictionaries to store multiple target variables
    y_train_dict = {
        'bs': y_bs_train,
        'ahbs': y_ahbs_train,
        'cw': y_cw_train
    }
    
    y_test_dict = {
        'bs': y_bs_test,
        'ahbs': y_ahbs_test,
        'cw': y_cw_test
    }

    joblib.dump(scaler, 'scaler.pkl')
    joblib.dump(feature_columns, 'feature_columns.pkl')
    
    return X_train_scaled, X_test_scaled, y_train_dict, y_test_dict, scaler, feature_columns

In [ ]:
def predict_and_add_to_test_data(models, test_data, feature_columns, scaler, error_type):

    # Create a copy of the test data to avoid modifying the original
    result_df = test_data
   
    # Extract features from test data as DataFrame (not as numpy array)
    X_test = test_data[feature_columns]
   
    # Scale the features using the pre-fitted scaler and convert back to DataFrame
    # This preserves feature names and prevents the warning
    X_test_scaled = pd.DataFrame(
        scaler.transform(X_test),
        columns=feature_columns,
        index=X_test.index
    )
   
    # Original value column name
    original_column = f'iv_{error_type}'
   
    # Generate predictions for each model
    for model_name, model in models.items():
        # Make predictions
        predictions = model.predict(X_test_scaled, batch_size=128, verbose=0)
       
        # Flatten predictions if needed
        if len(predictions.shape) > 1:
            predictions = predictions.flatten()
       
        # Add predictions to the dataframe
        column_name = f'iv_{error_type}_pred_{model_name}'
        result_df[column_name] = predictions
       
        # Calculate corrected value by adding the error prediction to the original value
        result_df[f'iv_{error_type}_corrected_{model_name}'] = result_df[original_column] - predictions
   
    return result_df

In [ ]:

###########################################
# PART 4: COMPLETE WORKFLOW
###########################################

if __name__ == "__main__":

    # Step 2: Prepare data
    X_train_scaled, X_test_scaled, y_train_dict, y_test_dict, scaler, feature_columns = prepare_data(option_only)
    
    # Step 3: Train models for each target variable
    models = {}
    results = {}
    
    # Dictionary to store all models
    all_models = {
        'bs': {},
        'ahbs': {},
        'cw': {}
    }
    
    # Train models for each error type
    for error_type in ['bs', 'ahbs', 'cw']:
        print(f"/n=== Training models for {error_type} error correction ===")
        
        # Get the appropriate training and test targets
        y_train = y_train_dict[error_type]
        y_test = y_test_dict[error_type]
        
        # Train each architecture
        for nn_type in ['NN3', 'NN4']:
            model_name = f"{nn_type}_{error_type}"
            print(f"/nTraining {model_name}...")
            
            # Create model
            model = create_nn_model(nn_type, X_train_scaled.shape[1])
            
            # Train and evaluate model
            history, mse = train_and_evaluate_model(
                model, X_train_scaled, y_train, X_test_scaled, y_test, epochs=50
            )
            
            # Store model and results
            all_models[error_type][nn_type] = model
            results[model_name] = {
                'mse': mse,
                'history': history
            }
            
            print(f"{model_name} Test MSE: {mse}")
            
            # Save model
            save_model(model, f"{model_name}_model_sameday.h5")
    
    # Step 4: Generate predictions on new test data for each error type
for error_type in ['bs', 'ahbs', 'cw']:
    print(f"\n=== Making predictions for {error_type} error correction ===")
    
    # Extract the models for this error type
    current_models = all_models[error_type]
    
    # Make predictions - use the accumulated DataFrame
    test_data_with_all_predictions = predict_and_add_to_test_data(
        current_models, test_data_with_all_predictions,feature_columns,  scaler, error_type
    )
    
    # Save intermediate results if desired
    test_data_with_all_predictions.to_csv(f'test_data_with_{error_type}_predictions_sameday.csv', index=False)
    
    # Display sample results from accumulated DataFrame
    print(f"\nSample of test data with {error_type} predictions:")
    display_columns = ['iv_' + error_type]
    for nn_type in ['NN3', 'NN4']:
        display_columns.extend([
            f'iv_{error_type}_pred_{nn_type}', 
            f'iv_{error_type}_corrected_{nn_type}'
        ])
    
    print(test_data_with_all_predictions[display_columns].head(5))
    save_model(model, f"{model_name}_model_sameday.h5")



/n=== Training models for bs error correction ===
/nTraining NN3_bs...


C:\Users\maxva\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
12772/12772 ━━━━━━━━━━━━━━━━━━━━ 8s 616us/step - loss: 0.0041 - learning_rate: 0.0100
Epoch 2/50
12772/12772 ━━━━━━━━━━━━━━━━━━━━ 8s 603us/step - loss: 4.2579e-04 - learning_rate: 0.0100
Epoch 3/50
12772/12772 ━━━━━━━━━━━━━━━━━━━━ 8s 602us/step - loss: 3.6727e-04 - learning_rate: 0.0100
Epoch 4/50
12772/12772 ━━━━━━━━━━━━━━━━━━━━ 8s 622us/step - loss: 2.8614e-04 - learning_rate: 0.0100
Epoch 5/50
12772/12772 ━━━━━━━━━━━━━━━━━━━━ 8s 656us/step - loss: 2.7854e-04 - learning_rate: 0.0100
Epoch 6/50
12772/12772 ━━━━━━━━━━━━━━━━━━━━ 8s 609us/step - loss: 2.4248e-04 - learning_rate: 0.0100
Epoch 7/50
12772/12772 ━━━━━━━━━━━━━━━━━━━━ 8s 596us/step - loss: 2.6145e-04 - learning_rate: 0.0100
Epoch 8/50
12772/12772 ━━━━━━━━━━━━━━━━━━━━ 8s 605us/step - loss: 2.6389e-04 - learning_rate: 0.0100
Epoch 9/50
12772/12772 ━━━━━━━━━━━━━━━━━━━━ 8s 631us/step - loss: 1.9776e-04 - learning_rate: 0.0100
Epoch 10/50
12772/12772 ━━━━━━━━━━━━━━━━━━━━ 8s 606us/step - loss: 2.0681e-04 - learning_rate: 

KeyboardInterrupt: 

Calculate IVRMSE

In [ ]:
def load_models(model_paths_dict):

    loaded_models = {}
    
    for error_type in model_paths_dict:
        loaded_models[error_type] = {}
        
        for model_name, path in model_paths_dict[error_type].items():
            try:
                # Handle Windows paths by using raw strings
                path = path.replace('\\', '/')  # Convert backslashes to forward slashes
                
                # Check if file exists
                if not os.path.exists(path):
                    print(f"✗ {error_type} {model_name}: File not found at {path}")
                    loaded_models[error_type][model_name] = None
                    continue
                
                # Load the model
                print(f"Loading {error_type} {model_name}...")
                model = load_model(path)
                
                # Simple check: can we access layers?
                num_layers = len(model.layers)
                print(f"✓ Model loaded successfully with {num_layers} layers")
                
                # Store loaded model
                loaded_models[error_type][model_name] = model
                
            except Exception as e:
                print(f"✗ Failed to load {error_type} {model_name}: {str(e)}")
                loaded_models[error_type][model_name] = None
    
    return loaded_models

# Example usage
if __name__ == "__main__":
    # Model paths dictionary with proper path handling for Windows
    # Use forward slashes or raw strings to avoid Unicode escape errors
    model_paths_dict = {
        'bs': {
            'NN3': r"C:/Users/maxva/OneDrive - Tilburg University/Msc. Data Science/Master Thesis/Code/Models/Neural Network SameDay\Without characteristics/NN3_bs_model_sameday.h5",
            'NN4': r"C:/Users/maxva/OneDrive - Tilburg University/Msc. Data Science/Master Thesis/Code/Models/Neural Network SameDay\Without characteristics/NN4_bs_model_sameday.h5"
        },
        'ahbs': {
            'NN3': r"C:/Users/maxva/OneDrive - Tilburg University/Msc. Data Science/Master Thesis/Code/Models/Neural Network SameDay\Without characteristics/NN3_ahbs_model_sameday.h5",
            'NN4': r"C:/Users/maxva/OneDrive - Tilburg University/Msc. Data Science/Master Thesis/Code/Models/Neural Network SameDay\Without characteristics/NN4_ahbs_model_sameday.h5"
        },
        'cw': {
            'NN3': r"C:/Users/maxva/OneDrive - Tilburg University/Msc. Data Science/Master Thesis/Code/Models/Neural Network SameDay\Without characteristics/NN3_cw_model_sameday.h5",
            'NN4': r"C:/Users/maxva/OneDrive - Tilburg University/Msc. Data Science/Master Thesis/Code/Models/Neural Network SameDay\Without characteristics/NN4_cw_model_sameday.h5"
        }
    }
    
    # Load all models
    loaded_models = load_models(model_paths_dict)
    
    # Print summary of loaded models
    print("\n=== Model Loading Summary ===")
    for error_type in loaded_models:
        for model_name in loaded_models[error_type]:
            status = "✓ Loaded" if loaded_models[error_type][model_name] is not None else "✗ Failed"
            print(f"{error_type} {model_name}: {status}")

Loading bs NN3...
✓ Model loaded successfully with 4 layers
Loading bs NN4...


✓ Model loaded successfully with 5 layers
Loading ahbs NN3...
✓ Model loaded successfully with 4 layers
Loading ahbs NN4...
✓ Model loaded successfully with 5 layers
Loading cw NN3...
✓ Model loaded successfully with 4 layers
Loading cw NN4...
✓ Model loaded successfully with 5 layers

=== Model Loading Summary ===
bs NN3: ✓ Loaded
bs NN4: ✓ Loaded
ahbs NN3: ✓ Loaded
ahbs NN4: ✓ Loaded
cw NN3: ✓ Loaded
cw NN4: ✓ Loaded


In [7]:

scaler_path = r"C:\Users\maxva\OneDrive - Tilburg University\Msc. Data Science\Master Thesis\Code\Models\Neural Network SameDay\Without characteristics\scaler.pkl"
scaler = joblib.load(scaler_path)

base_df = test_data_with_all_predictions

# 2) determine your features once
exclude_cols = [
    'iv_bs_error','iv_ahbs','iv_ahbs_error','iv_bs','iv_cw','iv_cw_error',
    'impl_volatility','moneyness_category','ID','date','new_id',
    'open_interest','option_price','prediction_horizon',
    'test_date','train_date','volume', 'iv_bs_pred_NN3', 'iv_bs_corrected_NN3', 'iv_bs_pred_NN4', 'iv_bs_corrected_NN4', 'iv_ahbs_pred_NN3', 'iv_ahbs_corrected_NN3', 'iv_ahbs_pred_NN4', 'iv_ahbs_corrected_NN4', 'iv_cw_pred_NN3', 'iv_cw_corrected_NN3', 'iv_cw_pred_NN4', 'iv_cw_corrected_NN4'
]
feature_columns = [c for c in base_df.columns if c not in exclude_cols]
print(feature_columns)
# 3) loop over error types _and_ models, but update the SAME DataFrame
df = base_df
for error_type in ['bs','ahbs','cw']:
    print(f"\n-- predicting corrections for {error_type} --")
    
    models_for_type = loaded_models[error_type]  # e.g. [nn3_model, nn4_model]
    
    # assume your helper loops through each model internally,
    # adding both iv_{error_type}_pred_NN3/_NN4 and iv_{error_type}_corrected_NN3/_NN4
    df = predict_and_add_to_test_data(
        models_for_type, df, feature_columns, scaler, error_type
    )
    
    # show a quick peek
    cols = ['impl_volatility', f'iv_{error_type}']
    for m in ['NN3','NN4']:
        cols += [f'iv_{error_type}_pred_{m}', f'iv_{error_type}_corrected_{m}']
    print(df[cols].head(3))


['cp_flag', 'stock_price', 'moneyness', 'time_to_expiry', 'strike_price', 'delta', 'gamma', 'vega', 'theta', 'rf']

-- predicting corrections for bs --
   impl_volatility     iv_bs  iv_bs_pred_NN3  iv_bs_corrected_NN3  \
0         0.217247  0.353798        0.142212             0.211586   
1         0.215223  0.353798        0.144742             0.209056   
2         0.212591  0.353798        0.147380             0.206419   

   iv_bs_pred_NN4  iv_bs_corrected_NN4  
0        0.141132             0.212666  
1        0.144622             0.209176  
2        0.147485             0.206313  

-- predicting corrections for ahbs --
   impl_volatility   iv_ahbs  iv_ahbs_pred_NN3  iv_ahbs_corrected_NN3  \
0         0.217247  0.350970          0.135346               0.215624   
1         0.215223  0.349422          0.137301               0.212120   
2         0.212591  0.348100          0.139177               0.208924   

   iv_ahbs_pred_NN4  iv_ahbs_corrected_NN4  
0          0.138737           

In [8]:
import numpy as np
from sklearn.metrics import mean_squared_error

def calculate_ivrmse(predictions_df, error_types=['bs', 'ahbs', 'cw'], models=['NN3', 'NN4']):
    results = {}
    
    # Calculate IVRMSE for each error type and model
    for error_type in error_types:
        orig_col = f'iv_{error_type}'
        
        # Calculate base IVRMSE (before correction)
        base_rmse = np.sqrt(mean_squared_error(predictions_df['impl_volatility'], predictions_df[orig_col]))
        results[f"{error_type}_base"] = base_rmse
        
        # Calculate IVRMSE for each model
        for model in models:
            corrected_col = f'iv_{error_type}_corrected_{model}'
            
            # Skip if corrected column doesn't exist
            if corrected_col not in predictions_df.columns:
                print(f"Warning: {corrected_col} column not found, skipping...")
                continue
                
            # Calculate IVRMSE for the corrected predictions
            corrected_diff = predictions_df['impl_volatility'] - predictions_df[corrected_col]
            corrected_rmse = np.sqrt(mean_squared_error(predictions_df['impl_volatility'], predictions_df[corrected_col]))
            results[f"{error_type}_{model}"] = corrected_rmse
            
            # Calculate improvement percentage
            improvement = (base_rmse - corrected_rmse) / base_rmse * 100
            results[f"{error_type}_{model}_improvement"] = improvement
    
    return results

# Example usage
if __name__ == "__main__":
    # Assuming test_data_with_all_predictions is your DataFrame with all predictions
    ivrmse_results = calculate_ivrmse(test_data_with_all_predictions)
    
    # Print results in a table format
    print("\n=== IVRMSE Results ===")
    print(f"{'Model':<15} {'IVRMSE':<10} {'Improvement':<12}")
    print("-" * 40)
    
    for error_type in ['bs', 'ahbs', 'cw']:
        base_key = f"{error_type}_base"
        if base_key in ivrmse_results:
            base_rmse = ivrmse_results[base_key]
            print(f"{error_type.upper():<15} {base_rmse:.6f}  {'(baseline)':<12}")
            
            for model in ['NN3', 'NN4']:
                model_key = f"{error_type}_{model}"
                imp_key = f"{error_type}_{model}_improvement"
                
                if model_key in ivrmse_results:
                    print(f"{model_key:<15} {ivrmse_results[model_key]:.6f}  {ivrmse_results[imp_key]:.2f}%")
            print("-" * 40)
    
    # Find best overall model
    model_keys = [k for k in ivrmse_results.keys() if not k.endswith('base') and not k.endswith('improvement')]
    if model_keys:
        best_model = min(model_keys, key=lambda k: ivrmse_results[k])
        print(f"\nBest overall model: {best_model} with IVRMSE = {ivrmse_results[best_model]:.6f}")
        
        base_key = f"{best_model.split('_')[0]}_base"
        imp_key = f"{best_model}_improvement"
        if base_key in ivrmse_results and imp_key in ivrmse_results:
            print(f"Improvement over baseline: {ivrmse_results[imp_key]:.2f}%")


=== IVRMSE Results ===
Model           IVRMSE     Improvement 
----------------------------------------
BS              0.150568  (baseline)  
bs_NN3          0.011421  92.41%
bs_NN4          0.011191  92.57%
----------------------------------------
AHBS            0.134068  (baseline)  
ahbs_NN3        0.011442  91.47%
ahbs_NN4        0.011282  91.58%
----------------------------------------
CW              0.128764  (baseline)  
cw_NN3          0.012688  90.15%
cw_NN4          0.013123  89.81%
----------------------------------------

Best overall model: bs_NN4 with IVRMSE = 0.011191
Improvement over baseline: 92.57%


IVRMSE Moneyness categories

In [12]:
def add_moneyness_categories(df):
    # Create a copy to avoid modifying the original dataframe
    result = df.copy()
    
    # Initialize moneyness_category column with NaN values
    result['moneyness_category'] = np.nan
    
    # Define conditions for each category
    dotmc_mask = (result['cp_flag'] == 1) & (result['moneyness'] >= 0.80) & (result['moneyness'] < 0.90)
    otmc_mask = (result['cp_flag'] == 1) & (result['moneyness'] >= 0.90) & (result['moneyness'] < 0.97)
    atm_mask = (result['moneyness'] >= 0.97) & (result['moneyness'] < 1.03)
    otmp_mask = (result['cp_flag'] == 0) & (result['moneyness'] >= 1.03) & (result['moneyness'] < 1.10)
    dotmp_mask = (result['cp_flag'] == 0) & (result['moneyness'] >= 1.10) & (result['moneyness'] <= 1.60)
    
    # Assign categories based on conditions
    result.loc[dotmc_mask, 'moneyness_category'] = 'DOTMC'
    result.loc[otmc_mask, 'moneyness_category'] = 'OTMC'
    result.loc[atm_mask, 'moneyness_category'] = 'ATM'
    result.loc[otmp_mask, 'moneyness_category'] = 'OTMP'
    result.loc[dotmp_mask, 'moneyness_category'] = 'DOTMP'
    
    return result

# Apply moneyness categorization
test_data_with_all_predictions = add_moneyness_categories(test_data_with_all_predictions)
print(f"Added moneyness categories to test_data_per_horizon")
    
    # Print sample data to verify
display_columns = ['cp_flag', 'moneyness', 'moneyness_category']
print(f"\nSample data for horizon:")
print(test_data_with_all_predictions[display_columns].head(5))

Added moneyness categories to test_data_per_horizon

Sample data for horizon:
   cp_flag  moneyness moneyness_category
0        1   0.996078                ATM
1        1   0.987564                ATM
2        1   0.979195                ATM
3        1   0.924360               OTMC
4        0   1.031652               OTMP


C:\Users\maxva\AppData\Local\Temp\ipykernel_24432\347831484.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'DOTMC' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  result.loc[dotmc_mask, 'moneyness_category'] = 'DOTMC'


In [ ]:
# Define the moneyness groups
moneyness_groups = ['DOTMC', 'OTMC', 'ATM', 'OTMP', 'DOTMP']

def analyze_ivrmse_by_moneyness(test_data):

    # 1. Group the test data by moneyness
    results = {}
    for group in moneyness_groups:
        group_data = test_data[test_data['moneyness_category'] == group]
        
        # Skip if no data in this group
        if len(group_data) == 0:
            print(f"Warning: No data found for moneyness group {group}")
            continue
            
        # Initialize results for this group
        results[group] = {}
        
        # 2. Calculate IVRMSE for each error type and model
        for error_type in ['bs', 'ahbs', 'cw']:
            # Column names
            orig_col = f'iv_{error_type}'
            
            # Skip if column doesn't exist
            if orig_col not in group_data.columns:
                print(f"Warning: Column {orig_col} not found, skipping...")
                continue
                
            # Calculate baseline IVRMSE
            base_rmse = np.sqrt(mean_squared_error(
                group_data['impl_volatility'], 
                group_data[orig_col]
            ))
            
            # Store baseline IVRMSE
            results[group][f"{error_type}_base"] = base_rmse
            
            # Calculate corrected IVRMSE for each model
            for model in ['NN3', 'NN4']:
                corrected_col = f'iv_{error_type}_corrected_{model}'
                
                # Skip if column doesn't exist
                if corrected_col not in group_data.columns:
                    print(f"Warning: Column {corrected_col} not found, skipping...")
                    continue
                    
                corrected_rmse = np.sqrt(mean_squared_error(
                    group_data['impl_volatility'], 
                    group_data[corrected_col]
                ))
                
                # Calculate improvement percentage
                improvement = (base_rmse - corrected_rmse) / base_rmse * 100
                
                # Store results
                results[group][f"{error_type}_{model}"] = corrected_rmse
                results[group][f"{error_type}_{model}_improvement"] = improvement
    
    return results

def format_results_table(results):
    # Prepare IVRMSE table data
    ivrmse_data = []
    for group, group_results in results.items():
        row = {'Moneyness Group': group}
        
        for key, value in group_results.items():
            if not key.endswith('improvement'):
                if key.endswith('base'):
                    # Format baseline columns
                    error_type = key.split('_')[0].upper()
                    row[f"{error_type} Base"] = value
                else:
                    # Format model columns
                    parts = key.split('_')
                    error_type = parts[0].upper()
                    model = parts[1]
                    row[f"{error_type} {model}"] = value
        
        ivrmse_data.append(row)
    
    # Prepare improvement percentage table data
    improvement_data = []
    for group, group_results in results.items():
        row = {'Moneyness Group': group}
        
        for key, value in group_results.items():
            if key.endswith('improvement'):
                # Format improvement columns
                parts = key.replace('_improvement', '').split('_')
                error_type = parts[0].upper()
                model = parts[1]
                row[f"{error_type} {model}"] = value
        
        improvement_data.append(row)
    
    # Create DataFrames
    ivrmse_df = pd.DataFrame(ivrmse_data)
    improvement_df = pd.DataFrame(improvement_data)
    
    # Sort columns for better readability
    ivrmse_cols = ['Moneyness Group']
    improvement_cols = ['Moneyness Group']
    
    for et in ['BS', 'AHBS', 'CW']:
        ivrmse_cols.extend([f"{et} Base", f"{et} NN3", f"{et} NN4"])
        improvement_cols.extend([f"{et} NN3", f"{et} NN4"])
    
    # Reorder columns if they exist
    ivrmse_df = ivrmse_df[[col for col in ivrmse_cols if col in ivrmse_df.columns]]
    improvement_df = improvement_df[[col for col in improvement_cols if col in improvement_df.columns]]
    
    return ivrmse_df, improvement_df

def find_best_models(results):
    best_models = []
    
    for group, group_results in results.items():
        for error_type in ['bs', 'ahbs', 'cw']:
            # Get baseline IVRMSE
            base_key = f"{error_type}_base"
            if base_key not in group_results:
                continue
                
            base_rmse = group_results[base_key]
            
            # Find best model for this error type
            models = [m for m in ['NN3', 'NN4'] if f"{error_type}_{m}" in group_results]
            if not models:
                continue
            
            # Find model with lowest IVRMSE    
            best_model = min(models, key=lambda m: group_results[f"{error_type}_{m}"])
            best_rmse = group_results[f"{error_type}_{best_model}"]
            improvement = group_results[f"{error_type}_{best_model}_improvement"]
            
            # Also get the values for the other model for comparison
            other_models = [m for m in models if m != best_model]
            other_model_data = {}
            if other_models:
                other_model = other_models[0]
                other_rmse = group_results[f"{error_type}_{other_model}"]
                other_improvement = group_results[f"{error_type}_{other_model}_improvement"]
                other_model_data = {
                    f'Other Model': other_model,
                    f'Other IVRMSE': other_rmse,
                    f'Other Improvement %': other_improvement
                }
            
            model_data = {
                'Moneyness Group': group,
                'Error Type': error_type.upper(),
                'Best Model': best_model,
                'Base IVRMSE': base_rmse,
                'Best IVRMSE': best_rmse,
                'Improvement %': improvement
            }
            
            # Add other model data if available
            model_data.update(other_model_data)
            
            best_models.append(model_data)
    
    return pd.DataFrame(best_models)

def print_formatted_tables(results):

    # Format results into DataFrames
    ivrmse_df, improvement_df = format_results_table(results)
    
    # Format IVRMSE table
    pd.set_option('display.float_format', '{:.6f}'.format)
    print("=" * 80)
    print("IVRMSE by Moneyness Group")
    print("=" * 80)
    print(ivrmse_df.to_string(index=False))
    
    # Format improvement table
    pd.set_option('display.float_format', '{:.2f}%'.format)
    print("\n" + "=" * 80)
    print("Improvement Percentage by Moneyness Group")
    print("=" * 80)
    print(improvement_df.to_string(index=False))
    
    # Find best models
    best_models_df = find_best_models(results)
    
    # Reset float format for mixed table
    pd.set_option('display.float_format', None)
    print("\n" + "=" * 80)
    print("Best Model by Moneyness Group and Error Type")
    print("=" * 80)
    # Format specific columns
    best_models_df['Base IVRMSE'] = best_models_df['Base IVRMSE'].map('{:.6f}'.format)
    best_models_df['Best IVRMSE'] = best_models_df['Best IVRMSE'].map('{:.6f}'.format)
    best_models_df['Improvement %'] = best_models_df['Improvement %'].map('{:.2f}%'.format)
    print(best_models_df.to_string(index=False))
    
    # Summary statistics
    print("\n" + "=" * 80)
    print("Summary Statistics")
    print("=" * 80)
    
    # Count best model occurrences
    model_counts = best_models_df['Best Model'].value_counts()
    print(f"Overall best model distribution: {dict(model_counts)}")
    
    # Average improvement by moneyness group
    print("\nAverage improvement by moneyness group:")
    # Convert percentage strings to numeric values
    best_models_df['Improvement_Numeric'] = pd.to_numeric(best_models_df['Improvement %'].str.rstrip('%'))
    
    # Group and calculate means
    avg_improvement = best_models_df.groupby('Moneyness Group')['Improvement_Numeric'].mean()
    
    # Handle the case where there's only one group (which returns a scalar)
    if isinstance(avg_improvement, pd.Series):
        # Sort if it's a Series with multiple values
        sorted_improvements = avg_improvement.sort_values(ascending=False)
        for group, imp in sorted_improvements.items():
            print(f"  {group}: {imp:.2f}%")
    else:
        # Just print the single value if it's a scalar
        group = best_models_df['Moneyness Group'].iloc[0]
        print(f"  {group}: {avg_improvement:.2f}%")

# Example usage:
results = analyze_ivrmse_by_moneyness(test_data_with_all_predictions)
print_formatted_tables(results)


IVRMSE by Moneyness Group
Moneyness Group  BS Base   BS NN3   BS NN4  AHBS Base  AHBS NN3  AHBS NN4  CW Base   CW NN3   CW NN4
          DOTMC 0.135785 0.011299 0.011230   0.135335  0.011855  0.011922 0.122504 0.014003 0.014875
           OTMC 0.134270 0.009091 0.008835   0.126841  0.008905  0.009309 0.125177 0.009858 0.010410
            ATM 0.140271 0.010410 0.009755   0.135411  0.010215  0.010211 0.131621 0.010889 0.010996
           OTMP 0.131159 0.009846 0.009727   0.129846  0.010106  0.009851 0.128342 0.011020 0.011246
          DOTMP 0.195498 0.015319 0.015249   0.142420  0.015224  0.014460 0.134349 0.016785 0.017231

Improvement Percentage by Moneyness Group
Moneyness Group  BS NN3  BS NN4  AHBS NN3  AHBS NN4  CW NN3  CW NN4
          DOTMC  91.68%  91.73%    91.24%    91.19%  88.57%  87.86%
           OTMC  93.23%  93.42%    92.98%    92.66%  92.12%  91.68%
            ATM  92.58%  93.05%    92.46%    92.46%  91.73%  91.65%
           OTMP  92.49%  92.58%    92.22%    92.41%  

In [ ]:
# get the table
dfm, _ = format_results_table(results)


# write to CSV (no index column)
dfm.to_csv('results_nn_s_f.csv', index=False)

Feature Importance

In [ ]:
all_models = {
    'bs': {},
    'ahbs': {},
    'cw': {}
}

def analyze_neural_network_weights(model, feature_names, model_name="Neural Network"):

    # Get weights from the first layer
    first_layer_weights = model.layers[0].get_weights()[0]  # Shape: (input_features, neurons)
    
    # Calculate importance as the mean absolute weight for each feature
    importance = np.mean(np.abs(first_layer_weights), axis=1)
    
    # Normalize to sum to 100 for easier interpretation
    importance = 100.0 * (importance / np.sum(importance))
    
    # Create DataFrame with feature names and importance
    importance_df = pd.DataFrame({
        'Feature': feature_names,
        'Importance': importance
    })
    
    # Sort by importance
    importance_df = importance_df.sort_values('Importance', ascending=False)
    
    # Print the top features
    print(f"Top 20 most important features for {model_name}:")
    print(importance_df.head(20))
    
    return importance_df

def plot_feature_importance(importance_df, model_name="Neural Network", top_n=30):

    # Select top N features
    plot_data = importance_df.head(top_n)
    
    # Create horizontal bar chart
    plt.figure(figsize=(12, 10))
    plt.barh(np.arange(len(plot_data)), plot_data['Importance'], align='center')
    plt.yticks(np.arange(len(plot_data)), plot_data['Feature'])
    plt.xlabel('Relative Importance (%)')
    plt.title(f'Top {top_n} Features - {model_name}')
    plt.tight_layout()
    plt.gca().invert_yaxis()  # Invert y-axis to have the most important feature at the top
    
    return plt.gcf()  # Return the figure for saving if needed

def calculate_permutation_importance(model, X, y, feature_names, n_repeats=10, random_state=42):

    # Function to predict using the neural network
    def predict_fn(X_data):
        return model.predict(X_data, verbose=0).flatten()
    
    # Calculate permutation importance using the sklearn function
    r = sklearn_permutation_importance(predict_fn, X, y, 
                                      n_repeats=n_repeats,
                                      random_state=random_state)
    
    # Create DataFrame with results
    perm_importance_df = pd.DataFrame({
        'Feature': feature_names,
        'Importance': r.importances_mean,
        'Std': r.importances_std
    })
    
    # Sort by importance
    perm_importance_df = perm_importance_df.sort_values('Importance', ascending=False)
    
    return perm_importance_df

def analyze_deeper_layers(model, layer_indices=None):

    if layer_indices is None:
        layer_indices = range(len(model.layers))
    
    layer_info = {}
    
    for i in layer_indices:
        layer = model.layers[i]
        weights = layer.get_weights()
        
        if len(weights) == 0:
            continue
            
        # For each layer, calculate weight statistics
        layer_info[i] = {
            'name': layer.name,
            'type': layer.__class__.__name__,
            'shape': [w.shape for w in weights],
            'weight_stats': {
                'mean': float(np.mean(weights[0])),
                'std': float(np.std(weights[0])),
                'min': float(np.min(weights[0])),
                'max': float(np.max(weights[0])),
                'sparsity': float(np.mean(np.abs(weights[0]) < 1e-10))
            }
        }
        
        if len(weights) > 1:  # Has bias
            layer_info[i]['bias_stats'] = {
                'mean': float(np.mean(weights[1])),
                'std': float(np.std(weights[1])),
                'min': float(np.min(weights[1])),
                'max': float(np.max(weights[1]))
            }
    
    return layer_info

def plot_weight_distributions(model, layer_indices=None):

    if layer_indices is None:
        layer_indices = range(len(model.layers))
    
    # Filter to keep only layers with weights
    layers_with_weights = []
    for i in layer_indices:
        if len(model.layers[i].get_weights()) > 0:
            layers_with_weights.append(i)
    
    if not layers_with_weights:
        print("No layers with weights found.")
        return
    
    n_layers = len(layers_with_weights)
    fig, axes = plt.subplots(n_layers, 1, figsize=(10, 3*n_layers))
    
    if n_layers == 1:
        axes = [axes]
    
    for i, layer_idx in enumerate(layers_with_weights):
        layer = model.layers[layer_idx]
        weights = layer.get_weights()[0]
        
        # Flatten weights for histogram
        flat_weights = weights.flatten()
        
        # Plot histogram
        axes[i].hist(flat_weights, bins=50)
        axes[i].set_title(f"Layer {layer_idx}: {layer.name} - Weight Distribution")
        axes[i].set_xlabel("Weight Value")
        axes[i].set_ylabel("Count")
    
    plt.tight_layout()
    return fig

def analyze_activation_patterns(model, X_sample, layer_indices=None):

    if layer_indices is None:
        layer_indices = range(len(model.layers))
    
    activation_models = {}
    activation_stats = {}
    
    for i in layer_indices:
        try:
            # Create a model that outputs the activations from this layer
            activation_model = tf.keras.Model(
                inputs=model.input,
                outputs=model.layers[i].output
            )
            
            # Get activations
            activations = activation_model.predict(X_sample, verbose=0)
            
            # Calculate activation statistics
            activation_stats[i] = {
                'layer_name': model.layers[i].name,
                'mean': float(np.mean(activations)),
                'std': float(np.std(activations)),
                'min': float(np.min(activations)),
                'max': float(np.max(activations)),
                'sparsity': float(np.mean(activations < 1e-10)),
                'shape': activations.shape
            }
        except:
            # Skip layers that don't support this analysis
            continue
    
    return activation_stats

if __name__ == "__main__":
    try:
        model = all_models['bs']['NN3']
    except KeyError:
        model = loaded_models['bs']['NN3']
    # Get feature names
    feature_names = feature_columns
    
    # 1. Analyze weights and show top features
    importance_df = analyze_neural_network_weights(model, feature_names, "NN3 - BS Error Correction")
    
    # 2. Plot feature importance
    fig = plot_feature_importance(importance_df, "NN3 - BS Error Correction", top_n=30)
    plt.savefig("NN3_bs_feature_importance.png")
    plt.close()
    
    # 3. Analyze deeper layers
    layer_info = analyze_deeper_layers(model)
    print("\n=== Layer Information ===")
    for layer_idx, info in layer_info.items():
        print(f"\nLayer {layer_idx}: {info['name']} ({info['type']})")
        print(f"  Shape: {info['shape']}")
        print(f"  Weight stats: Mean={info['weight_stats']['mean']:.4f}, "
              f"Std={info['weight_stats']['std']:.4f}")
        if 'bias_stats' in info:
            print(f"  Bias stats: Mean={info['bias_stats']['mean']:.4f}, "
                  f"Std={info['bias_stats']['std']:.4f}")
    
    # 4. Plot weight distributions
    weight_fig = plot_weight_distributions(model)
    plt.savefig("NN3_bs_weight_distributions.png")
    plt.close()
    

Top 20 most important features for NN3 - BS Error Correction:
          Feature  Importance
8           theta   20.366203
4    strike_price   14.395611
1     stock_price   14.167562
7            vega   10.933832
5           delta    9.789684
6           gamma    8.356256
3  time_to_expiry    8.183577
0         cp_flag    6.330551
2       moneyness    5.698659
9              rf    1.778068

=== Layer Information ===

Layer 0: dense_27 (Dense)
  Shape: [(10, 128), (128,)]
  Weight stats: Mean=0.0458, Std=1.2068
  Bias stats: Mean=-0.3419, Std=0.9816

Layer 1: dense_28 (Dense)
  Shape: [(128, 64), (64,)]
  Weight stats: Mean=-0.1313, Std=0.2998
  Bias stats: Mean=-0.1656, Std=0.0772

Layer 2: dense_29 (Dense)
  Shape: [(64, 32), (32,)]
  Weight stats: Mean=-0.0744, Std=0.2831
  Bias stats: Mean=-0.4025, Std=0.4794

Layer 3: dense_30 (Dense)
  Shape: [(32, 1), (1,)]
  Weight stats: Mean=0.0046, Std=0.2329
  Bias stats: Mean=-0.0214, Std=0.0000


In [46]:
sample_df = option_only.sample(n=10, random_state=42)


In [47]:
sample_df

,impl_volatility,cp_flag,stock_price,moneyness,time_to_expiry,strike_price,delta,gamma,vega,theta,rf,iv_ahbs,iv_ahbs_error,iv_bs,iv_bs_error,iv_cw,iv_cw_error
117086,0.234482,1,153.58500,0.903441,109,170.0,0.236069,0.015602,25.841560,-10.622040,0.0234,0.310503,0.076021,0.353798,0.119316,0.309728,0.075246
859670,0.575511,1,271.44855,0.835226,32,325.0,0.136164,0.004826,17.155330,-56.423970,0.0162,0.352766,-0.222745,0.353798,-0.221713,0.443834,-0.131677
204833,0.257683,1,29.72500,0.928906,232,32.0,0.425090,0.063260,9.311382,-2.143478,0.0274,0.321713,0.064030,0.353798,0.096115,0.299349,0.041666
1549824,0.197926,1,138.72000,0.894968,46,155.0,0.064338,0.013023,6.233315,-5.350260,0.0369,0.333355,0.135429,0.353798,0.155872,0.347881,0.149955
1270474,0.396995,0,136.17195,1.184104,172,115.0,-0.219291,0.008073,27.420670,-10.713180,0.0260,0.317334,-0.079661,0.353798,-0.043197,0.335632,-0.061363
1326742,0.605911,0,27.52500,1.019444,78,27.0,-0.383516,0.048552,4.949155,-6.649651,0.0376,0.316321,-0.289590,0.353798,-0.252113,0.306363,-0.299548
869960,0.264442,1,234.25500,0.851836,57,275.0,0.058229,0.004818,10.505050,-9.113153,0.0163,0.337809,0.073367,0.353798,0.089356,0.380165,0.115723
1295260,0.445018,0,15.75000,1.050000,147,15.0,-0.403819,0.089536,3.753302,-1.872832,0.0304,0.291049,-0.153969,0.353798,-0.091220,0.305216,-0.139802
822677,0.475291,1,169.41500,0.868795,22,195.0,0.153299,0.011796,9.983296,-39.390260,0.0107,0.350550,-0.124741,0.353798,-0.121493,0.409967,-0.065324
1633823,0.304587,0,28.75000,1.150000,78,25.0,-0.132590,0.053588,2.842366,-1.827551,0.0409,0.359939,0.055352,0.353798,0.049211,0.374959,0.070372


In [ ]:
def manual_permutation_importance(model, X: np.ndarray, y: np.ndarray,
                                  feature_names: list[str],
                                  n_repeats: int = 10,
                                  random_state: int = 42) -> pd.Series:

    rng = np.random.RandomState(random_state)
    baseline_preds = model.predict(X, verbose=0).flatten()
    baseline_score = mean_squared_error(y, baseline_preds)

    importances = []
    for idx, col in enumerate(feature_names):
        scores = []
        for _ in range(n_repeats):
            X_permuted = X.copy()
            X_permuted[:, idx] = rng.permutation(X_permuted[:, idx])
            permuted_preds = model.predict(X_permuted, verbose=0).flatten()
            score = mean_squared_error(y, permuted_preds)
            scores.append(score - baseline_score)
        importances.append(np.mean(scores))

    return pd.Series(importances, index=feature_names).sort_values(ascending=False)

def plot_importance(importance: pd.Series, top_n: int = 20, title: str = "Feature Importance"):
    """
    Plot horizontal bar chart for the top_n features.
    """
    data = importance.head(top_n).sort_values()
    plt.figure(figsize=(8, max(4, 0.2 * top_n)))
    data.plot.barh()
    plt.xlabel('Importance (Increase in MSE)')
    plt.title(title)
    plt.tight_layout()
    plt.show()

# Prepare data once


# Load pre-saved feature columns (if different than current feature_names)
feature_columns = joblib.load(r'C:\Users\maxva\OneDrive - Tilburg University\Msc. Data Science\Master Thesis\Code\Models\Neural Network SameDay\Without Characteristics\feature_columns.pkl')

# No manual slicing here
_, X_test_scaled, _, y_test_dict, _, feature_columns = prepare_data(sample_df)



# Access the specific model you want
model = loaded_models['bs']['NN3']

# Select the correct target (y) for 'bs'
y_test = y_test_dict['bs']

# Compute permutation importance
imp_perm = manual_permutation_importance(model, X_test_scaled, y_test.values, feature_columns)

# Plot the results
title = "BS NN3 Combined Dataset"
plot_importance(imp_perm, top_n=20, title=title)


KeyboardInterrupt: 